In [1]:
import os
import sys
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI
import re 
from langchain_community.graphs import Neo4jGraph
import dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import numpy as np 
import pandas as pd
# from project import web_crawl
import subprocess
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

USER_AGENT environment variable not set, consider setting it to identify your requests.


# connection to API's 

In [2]:

#LLM API's 
dotenv.load_dotenv()
llm=AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    temperature=0,
    
)

graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
)


In [3]:
# empty graph database
# DElete everything from the database 
delete_cypher=""" 
Match(n)
DETACH DELETE n
"""
graph.query(delete_cypher)

[]

# extract information from weburl


In [4]:
from langchain_core.documents import Document
from helpers.clean_data import clean_text
url_input=("https://en.wikipedia.org/wiki/Elon_Musk")
loader=WebBaseLoader([url_input])
data=loader.load().pop().page_content
data
    

'\n\n\n\nElon Musk - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life and education\n\n\n\n\nToggle Early life and education subsection\n\n\n\n\n\n1.1\nChildhood and family\n\n\n\n\n\n\n\n\n1.2\nEducation\n

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

data=clean_text(data)
documents=[Document(page_content=data)]
# print(documents)
splitter=RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=100)
smaller_doc=splitter.split_documents(documents)
len(smaller_doc)

78

# setting up prompt template example

In [6]:
template="""
you are an intelliegnt bot who can analyze any text with person information {doc} , 
your job is to read and analyse the information and create a json dictionary.
dictionary has the followinmg key:
Name:
Company:
Skills:
output must be json nothing else
"""

In [7]:
prompt=PromptTemplate(template=template,input_variables=['doc'])
llmchain=LLMChain(llm=llm,prompt=prompt)

C:\Users\26amr\AppData\Local\Temp\ipykernel_42992\2599093318.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llmchain=LLMChain(llm=llm,prompt=prompt)


In [8]:
for doc in smaller_doc[:2]:
    response=llmchain.invoke({'doc':doc.page_content})
    print(response['text'])
    

{
  "Name": "Elon Musk",
  "Company": [
    "SpaceX",
    "Tesla Inc",
    "X",
    "Twitter"
  ],
  "Skills": [
    "Entrepreneurship",
    "Engineering",
    "Innovation",
    "Leadership",
    "Technology"
  ]
}
{
  "Name": "Elon Reeve Musk",
  "Company": [
    "Tesla Inc",
    "X Corp",
    "The Boring Company",
    "xAI",
    "Neuralink",
    "OpenAI",
    "SpaceX"
  ],
  "Skills": [
    "Business Management",
    "Entrepreneurship",
    "Engineering",
    "Innovation",
    "Leadership",
    "Technology"
  ]
}


# from the predefined prompts

In [9]:
from helpers.prompts import graphPrompt
results=[]
for doc in smaller_doc[:15]:
    results.append(graphPrompt(doc.page_content))
results

{
    "entities": [
        {
            "label": "Elon Musk",
            "id": "person1",
            "role": "Entrepreneur",
            "description": "South African-born businessman and entrepreneur"
        },
        {
            "label": "SpaceX",
            "id": "organization1",
            "role": "Founder",
            "description": "Founder, CEO, and chief engineer of SpaceX"
        },
        {
            "label": "Tesla Inc.",
            "id": "organization2",
            "role": "CEO and Product Architect",
            "description": "CEO and product architect of Tesla Inc."
        },
        {
            "label": "X",
            "id": "organization3",
            "role": "Owner, CTO, and Executive Chairman",
            "description": "Owner, CTO, and Executive Chairman of X (formerly Twitter)"
        }
    ],
    "relationships": [
        {
            "label": "Founded",
            "id": "relationship1",
            "type": "Founded",
            "source

[{'entities': [{'label': 'Elon Musk',
    'id': 'person1',
    'role': 'Entrepreneur',
    'description': 'South African-born businessman and entrepreneur'},
   {'label': 'SpaceX',
    'id': 'organization1',
    'role': 'Founder',
    'description': 'Founder, CEO, and chief engineer of SpaceX'},
   {'label': 'Tesla Inc.',
    'id': 'organization2',
    'role': 'CEO and Product Architect',
    'description': 'CEO and product architect of Tesla Inc.'},
   {'label': 'X',
    'id': 'organization3',
    'role': 'Owner, CTO, and Executive Chairman',
    'description': 'Owner, CTO, and Executive Chairman of X (formerly Twitter)'}],
  'relationships': [{'label': 'Founded',
    'id': 'relationship1',
    'type': 'Founded',
    'source': 'person1',
    'target': 'organization1',
    'description': 'Elon Musk founded SpaceX, a spaceflight services company, in 2002'},
   {'label': 'CEO of',
    'id': 'relationship2',
    'type': 'CEO of',
    'source': 'person1',
    'target': 'organization2',
   

In [10]:
for res in results:
    print(res)


{'entities': [{'label': 'Elon Musk', 'id': 'person1', 'role': 'Entrepreneur', 'description': 'South African-born businessman and entrepreneur'}, {'label': 'SpaceX', 'id': 'organization1', 'role': 'Founder', 'description': 'Founder, CEO, and chief engineer of SpaceX'}, {'label': 'Tesla Inc.', 'id': 'organization2', 'role': 'CEO and Product Architect', 'description': 'CEO and product architect of Tesla Inc.'}, {'label': 'X', 'id': 'organization3', 'role': 'Owner, CTO, and Executive Chairman', 'description': 'Owner, CTO, and Executive Chairman of X (formerly Twitter)'}], 'relationships': [{'label': 'Founded', 'id': 'relationship1', 'type': 'Founded', 'source': 'person1', 'target': 'organization1', 'description': 'Elon Musk founded SpaceX, a spaceflight services company, in 2002'}, {'label': 'CEO of', 'id': 'relationship2', 'type': 'CEO of', 'source': 'person1', 'target': 'organization2', 'description': 'Elon Musk is the CEO and product architect of Tesla Inc.'}, {'label': 'Owner of', 'id'

In [11]:
# empty neo
graph.query(delete_cypher)

[]

# inject data into neo4j

In [12]:
from neo4j import GraphDatabase


In [13]:
driver=GraphDatabase.driver(uri=os.getenv('NEO4J_URI_ONLINE'),auth=(os.getenv('NEO4J_USERNAME_ONLINE'),os.getenv('NEO4J_PASSWORD_ONLINE')))

def sanitize_label(label):
    return re.sub(r"[^\w]", "_", label)

def insert_entities(session,entities):
    for entity in entities:
        query=f"""
              MERGE (n:{sanitize_label(entity['label'])} {{
               id:$id,
               role:$role,
               description:$description
              }})

              """
        session.run(query,label=entity['label'], id=entity['id'],role=entity['role'],description=entity['label'])
        print(f'Inserted entity: {entity["id"]}')
    
def insert_relationship(session,relationships):
    for relation in relationships:
        query = f"""
        MATCH (a {{id: $source}}), (b {{id: $target}})
        MERGE (a)-[:{sanitize_label(relation['type'])} {{
            id: $id,
            description: $description
        }}]->(b)
        """
        session.run(query,source=relation['source'],target=relation['target'],id=relation['id'],description=relation['description'])
        print(f'Inserted relationship: {relation["id"]}')


In [14]:
with driver.session() as session:
    for result in results:
        try:
            session.execute_write(insert_entities,result['entities'])
            session.execute_write(insert_relationship,result['relationships'])
        except Exception as e:
            print(f'Error {e}')
driver.close()

Inserted entity: person1
Inserted entity: organization1
Inserted entity: organization2
Inserted entity: organization3
Inserted relationship: relationship1
Inserted relationship: relationship2
Inserted relationship: relationship3
Inserted entity: person1
Inserted entity: organization1
Inserted entity: organization2
Inserted entity: organization3
Inserted entity: organization4
Inserted entity: organization5
Inserted entity: organization6
Inserted entity: organization7
Inserted entity: organization8
Inserted entity: organization9
Inserted entity: organization10
Inserted entity: organization11
Inserted entity: organization12
Inserted entity: organization13
Inserted entity: organization14
Inserted entity: organization15
Inserted entity: person2
Inserted entity: person3
Inserted entity: person4
Inserted entity: person5
Inserted entity: person6
Inserted entity: person7
Inserted entity: person8
Inserted entity: person9
Inserted entity: person10
Inserted entity: event1
Error 'role'
Inserted ent

In [15]:
result

{'entities': [{'label': 'Elon Musk',
   'id': 'person1',
   'role': 'Entrepreneur',
   'description': 'Visionary entrepreneur and business magnate'},
  {'label': 'Hyperloop',
   'id': 'event1',
   'role': 'Hyperloop',
   'description': 'Verbal government approval to build a Hyperloop from New York City to Washington DC'},
  {'label': 'The Boring Company',
   'id': 'organization1',
   'role': 'Founder',
   'description': 'Founder of The Boring Company'},
  {'label': 'Hawthorne Tunnel Project',
   'id': 'event2',
   'role': 'Hawthorne Tunnel Project',
   'description': 'Tunnel project to Hawthorne was discontinued in 2022'},
  {'label': 'OpenAI',
   'id': 'organization2',
   'role': 'Co-founder',
   'description': 'Co-founded OpenAI, a not-for-profit AI research company'},
  {'label': 'xAI',
   'id': 'organization3',
   'role': 'Founder',
   'description': 'Founder of xAI, an artificial intelligence company'},
  {'label': 'Tham Luang Cave Rescue',
   'id': 'event3',
   'role': 'Tham Luan